<a href="https://colab.research.google.com/github/feefaf/Projet_personnel/blob/main/Decision_tree_maker_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESISION TREE MAKER

## I First part - Data importation and manipulation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/data_crime.csv"
data = pd.read_csv(path, sep = ",", header = 0)
data

,Annee,States,Murder and nonnegligent manslaughter,Rape,Aggravated assault,Simple assault,Intimidation,Robbery,Burglary,Larceny-theft,Motor vehicle theft,Arson,Destruction/damage/vandalism
0,2004,Alabama,0,0,0,3,1,0,0,0,0,0,1
1,2004,Alaska,0,0,3,1,5,0,0,0,0,0,1
2,2004,Arizona,0,0,25,55,108,2,5,0,0,1,88
3,2004,Arkansas,0,1,10,22,15,2,11,9,0,0,18
4,2004,California,0,0,246,350,467,44,24,4,0,7,502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,2019,Virginia,0,0,12,55,55,2,0,4,0,0,47
795,2019,Washington,1,3,82,164,259,16,6,16,0,1,97
796,2019,West Virginia,0,0,3,7,8,1,3,2,0,0,4
797,2019,Wisconsin,0,3,5,25,15,1,2,6,0,0,14


In [ ]:
dataToRatio = data.groupby('States').sum()
dataToRatio = dataToRatio.drop('Annee', axis = 1)
dataToRatio = dataToRatio.reset_index(level=0)

In [ ]:
CrimeKeys = list(dataToRatio.keys())
CrimeKeys.pop(0)
StatesList = list(dataToRatio['States'])

In [ ]:
#Creation of the dictionary where the calculus are gonna be based on
RatioDict = {}
for i in range(len(StatesList)):
    tempDict = {}
    for crimes in CrimeKeys:
        #print(dataToRatio[crimes][i]/sum(list(dataToRatio[crimes])))
        tempDict[crimes] = (dataToRatio[crimes][i]/100000)
    RatioDict[str(StatesList[i])] = tempDict 


## Stats found on majoritly on:
https://bjs.ojp.gov/content/pub/pdf/cv19.pdf

https://bjs.ojp.gov/content/pub/pdf/fvv.pdf

https://bjs.ojp.gov/content/pub/pdf/wo.pdf

and other sources

### Great stats:

#### Rape victims:

91% female  9% male

#### Aggravated assault

63% female  37% male

#### simple assault

88% female  12% male

#### robbery
61% female 39% male

#### bulgary

house: 50% appartment 50%
if appartement are you on first floor or second floor?

yes +10% no nothing

do you have a dog?

yes -10% no nothing

are you planing to place cameras in your house?

yes -10% no nothing

do you have a gun for self defense?

yes -10% no nothing
/or -5%for
Larency theft 


#### Larency theft

house: 50% appartment 50%

if appartement are you on first floor or second floor?

yes +10% no nothing

do you have a dog?

yes -10% no nothing

are you planing to place cameras in your house?

yes -10% no nothing


#### motor vehicule theft

do you have a car?

yes 100% no 0%

#### arson
will your house made in majority out of wood?

yes 70% no 30%

will you live near a forest?

yes +10% no nothing

#### destruction damage vandalism
do you have a car?

yes 65% no 35%

## II Creation of the tree

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def scoreGenerator(state,ProbArr):
    score = 0
    Max = 0
    index = 0
    for i in range(len(CrimeKeys)):
        if Max < RatioDict[state][CrimeKeys[i]]*ProbArr[i]:
            Max = RatioDict[state][CrimeKeys[i]]*ProbArr[i]
            Index = i
        score += RatioDict[state][CrimeKeys[i]]*ProbArr[i]
    return "---------------------------------------------------------\nThere is a "+str(score*100)+"% chance that something really bad happen to you\nAnd you are most likely to be a victim of "+CrimeKeys[index]+" with a probability of "+str(Max*100)+"%"


def binaryMaker(string):
    if string == "y":
        return 1
    else:
        return 0

stateMovingWidget = widgets.Box(
    [
        widgets.Label(value='Wich state are you planing to move in: '),
        widgets.Dropdown(
    options=['Alabama','Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah','Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
    value='California',
    disabled=False,
)
    ]
)
    
def decisionTree():
    display(stateMovingWidget)
    stateMoving = stateMovingWidget.children[1].value


    #RAPE BLOC
    menRape = 0.09
    womenRape = 0.91

    familly = input('Are you moving with your family? y/n : ')
    if familly == "n":
        alone = input("Are you moving alone? y/n : ")
        if alone == "y":
            sexStr = input("gender? m/f : ")
            if sexStr == "m":
                sexM = 1
                sexF = 0
                rapeValue = menRape
            else:
                sexM = 0
                sexF = 1
                rapeValue = womenRape
        else:
            sexM = int(input("Enter the number of men you are moving with : "))        
            sexF = int(input("Enter the number of women you are moving with : "))
            rapeValue = (menRape*sexM+womenRape*sexF)/(sexM+sexF)
    else:
        sexM = int(input("Enter the number of men older than 18 years old you are moving with : "))        
        sexF = int(input("Enter the number of women older than 18 years old you are moving with : "))
        rapeValue = (menRape*sexM+womenRape*sexF)/(sexM+sexF)

    #AGGRAVATED ASSAULT BLOC
    menAA = 0.37
    womenAA = 0.63

    AAValue = (menAA*sexM+womenAA*sexF)/(sexM+sexF)

    #SIMPLE ASSAULT BLOC
    menSA = 0.12
    womenSA = 0.88

    SAValue =(menSA*sexM+womenSA*sexF)/(sexM+sexF)

    #ROBERY BLOC
    menRobery = 0.39
    womenRobery = 0.61

    roberyValue = (menRobery*sexM+womenRobery*sexF)/(sexM+sexF)

    #BURGLARY AND LARENCY THEFT AND ARSON BLOC
    houseValue = 0.5
    appValue = 0.5


    cameras = input("Are you planning to place cameras? y/n : ")
    camerasValue = binaryMaker(cameras)
    Gun = input("do you have a gun for self defense? y/n : ")
    gunValue = binaryMaker(Gun)
    HorA = input("Will you live in house or in an appartment? h/a : ")
    if HorA == "a":
        floor = input("Will you live on the first or second floor? y/n : ")
        floorValue = binaryMaker(floor)
        burglaryValue = (appValue-(0.1*gunValue)-(0.1*camerasValue)+(0.10*floorValue))
        larencyTheftValue = (appValue-(0.05*gunValue)-(0.10*camerasValue)+(0.10*floorValue))
        arsonValue = appValue
    else:
        dog = input("Do you have a dog that can protect your house? y/n : ")
        dogValue = binaryMaker(dog)
        wood = input("Is your future house made in majority out of wood? y/n : ")
        woodValue = binaryMaker(wood)
        burglaryValue = (houseValue-(0.10*gunValue)-(0.10*camerasValue)-(0.10*dogValue))
        larencyTheftValue = (houseValue-(0.05*gunValue)-(0.10*camerasValue)-(0.15*dogValue))
        arsonValue = houseValue+(0.20*woodValue)

    #MOTOR VEHICULE THEFT AND DESTRUCTION/VANDALISM BLOC

    motorVehicule = input("Do you have a motor vehicule? y/n : ")
    motorVehiculeValue = binaryMaker(motorVehicule)
    vandalismValue = (0.65*motorVehiculeValue)+(0.35*(1-motorVehiculeValue))



    ProbabilityArray = [0.5,rapeValue,AAValue,SAValue,0.5,roberyValue,burglaryValue,larencyTheftValue,motorVehiculeValue,arsonValue,vandalismValue]

    return stateMoving, ProbabilityArray
    

In [ ]:
#If the inputs field are too long, make sure you have the version of chrome or firefox
state, probarr = decisionTree()

Box(children=(Label(value='Wich state are you planing to move in: '), Dropdown(index=4, options=('Alabama', 'A…

Are you moving with your family? y/n : n
Are you moving alone? y/n : y
gender? m/f : m
Are you planning to place cameras? y/n : y
do you have a gun for self defense? y/n : n
Will you live in house or in an appartment? h/a : h
Do you have a dog that can protect your house? y/n : y
Is your future house made in majority out of wood? y/n : y
Do you have a motor vehicule? y/n : y


## III Launch this cell to get the result

In [ ]:
print(scoreGenerator(state,probarr))

---------------------------------------------------------
There is a 4.44304% chance that something really bad happen to you
And you are most likely to be a victim of Murder and nonnegligent manslaughter with a probability of 1.9668999999999999%
